# "Austrian Covid data exploration"
> "Exploring the latest figures of COVID19 in Austria"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Ifan Johnston
- categories: [covid]

# Importing the data

The data used in this post can be found at https://www.data.gv.at/covid-19/. After downloading the CSV file called `CovidFaelle_Timeline.csv`, we need to do some cleaning of the date column and split off some specific sets - the numbers for Austria, Volarlberg, Tirol and Wien. We also need to format the decimal numbers since the CSV file uses a ',' instead of a '.' as decimal.

In [1]:
#collapse
import pandas as pd


at_data = pd.read_csv("../data/CovidFaelle_Timeline.csv", sep=',')
current_date = at_data.iloc[-1,0]
all_austria = at_data.query("BundeslandID==10").sort_values(by='Time').sort_values(by='Time')
bund_ids = [8, 7, 9, 10]
vol_tyr_wien_at = at_data.query("BundeslandID==@bund_ids")
print(f"Note that the latest date in this data is {current_date}.")

Note that the latest date in this data is 2021-03-22.


In [2]:
#collapse
from ipywidgets import Button, Layout, jslink, AppLayout

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

latest_rate_vbg = at_data.query("Bundesland=='Vorarlberg'").iloc[-7:,7]
latest_rate_aus = at_data.query("Bundesland=='Österreich'").iloc[-7:,7]
header_button = create_expanded_button(f"7 day rate for districts in Austria", 'danger')
left_button = create_expanded_button(f"Vorarlberg {str(round(list(latest_rate_vbg)[-1]))}", '')
left_button.style.button_color='white'
right_button = create_expanded_button(f"Austria {str(round(list(latest_rate_aus)[-1]))}", '')
right_button.style.button_color='white'
footer_button = create_expanded_button(f"", 'danger')

AppLayout(header=header_button,
          left_sidebar=left_button,
          center=None,
          right_sidebar=right_button,
          footer=footer_button)

AppLayout(children=(Button(button_style='danger', description='7 day rate for districts in Austria', layout=La…

In [3]:
#collapse
vbg_change = round(list(latest_rate_vbg)[-1] - list(latest_rate_vbg)[0]) / list(latest_rate_vbg)[0]
aus_change = round(list(latest_rate_aus)[-1] - list(latest_rate_aus)[0]) / list(latest_rate_aus)[0]
week_trend_vbg = 'Down' if (round(list(latest_rate_vbg)[0] - list(latest_rate_vbg)[-1])) > 0 else f'Up ({vbg_change:.1%})'
week_trend_aus = 'Down' if (round(list(latest_rate_aus)[0] - list(latest_rate_aus)[-1])) > 0 else f'Up ({aus_change:.1%})'
header_button = create_expanded_button(f"Percentage change since 7 days ago", 'danger')
left_button = create_expanded_button(f"Vorarlberg: {week_trend_vbg}", '')
left_button.style.button_color='white'
right_button = create_expanded_button(f"Austria: {week_trend_aus}", '')
right_button.style.button_color='white'
footer_button = create_expanded_button(f"", 'danger')

AppLayout(header=header_button,
          left_sidebar=left_button,
          center=None,
          right_sidebar=right_button,
          footer=footer_button)

AppLayout(children=(Button(button_style='danger', description='Percentage change since 7 days ago', layout=Lay…

# Plotting the data

First we have the number of cases per 100,000 population across the whole of Austria. After a very sharp rise in cases during the end of October and the middle of November, the number of new cases have fallen due to the second lockdown. At the time of writting, the cases seem to have levelled off a little - though the Austrian government announced a third lockdown lasting a month starting from the 26th of December, in the expectation that cases will continue to rise over the festive period.

In [4]:
#collapse
import altair as alt

alt.Chart(all_austria).mark_bar(
    color='red',
    opacity=1,
    size=1
).encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("SiebenTageInzidenzFaelle:Q",
            axis=alt.Axis(title='Cases per 100k')),
).configure_axis(grid=True).configure_view(strokeWidth=0.1).properties(
    title='Number of cases per 100,000 in Austria', width=800
)

alt.Chart(...)

In [5]:
#collapse
bars = alt.Chart(all_austria).mark_bar(
    color='red',
    opacity=1,
    size=2
).encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("AnzahlFaelle:Q", axis=alt.Axis(title='New cases')),
)

line = alt.Chart(all_austria).mark_line(
    color='blue',
    opacity=0.75,
    size=1.5,
).transform_window(
    rolling_mean='mean(AnzahlFaelle)',
    frame=[0, 7]
).encode(x='yearmonthdate(Time):T',
         y='rolling_mean:Q'
         ).properties(title='New cases per day with rolling mean', width=800)

alt.layer(bars, line, data=all_austria)

alt.LayerChart(...)

Next we have the 7 day incidence rate for states of Vorarlberg, Tirol and Wien compared to all of Austria.

In [6]:
#collapse
leg_selection = alt.selection_multi(fields=['Bundesland'], bind='legend')
brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(at_data).mark_line().encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("SiebenTageInzidenzFaelle:Q", axis=alt.Axis(title='Incidence rate')),
    tooltip='Bundesland',
    color='Bundesland',
    opacity=alt.condition(leg_selection, alt.value(2), alt.value(0.1))
).add_selection(leg_selection).properties(width=800)

upper = base.encode(
    alt.X('yearmonthdate(Time):T',axis=alt.Axis(title=''),
          scale=alt.Scale(domain=brush))
).properties(title='7 day incidence rate for states in Austria')

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)

Finally we have the 

In [7]:
#collapse
brush = alt.selection(type='interval', encodings=['x'])
states = at_data['Bundesland'].unique()
states.sort()

selection = alt.selection_single(
    name='Select',
    fields=['Bundesland'],
    init={'Bundesland': 'Vorarlberg'},
    bind={'Bundesland': alt.binding_select(options=states)}
)

# scatter plot, modify opacity based on selection
bars = alt.Chart(at_data).mark_bar().add_selection(
    selection
).encode(
    x=alt.X("yearmonthdate(Time):T", axis=alt.Axis(title='Date')),
    y=alt.Y("SiebenTageInzidenzFaelle:Q", axis=alt.Axis(title='Incidence rate')),
    tooltip='SiebenTageInzidenzFaelle:Q',
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).properties(title=f'7 day incidence rate of individual states vs rolling mean across Austria', width=800)

line = alt.Chart(all_austria).mark_line(
    color='red',
    size=2,
).transform_window(
    rolling_mean='mean(SiebenTageInzidenzFaelle)',
    frame=[0, 7]
).encode(
    x='yearmonthdate(Time):T',
    y='rolling_mean:Q'
)

base = alt.layer(bars, line)
upper_bars = bars.encode(
    alt.X('yearmonthdate(Time):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)
upper_line = line.encode(
    alt.X('yearmonthdate(Time):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

upper = alt.layer(upper_bars, upper_line)

lower = base.properties(
    height=60
).add_selection(brush)
upper & lower

alt.VConcatChart(...)

Choose from the above dropdown menu to view the different states.